In [1]:
import torch
from utils import *       #changed
from models import TransformerLM, TransformerConditionedLM
import pickle
from imageio import imread
from PIL import Image
import torch.nn.functional as F

In [27]:
import yaml
# /net/papilio/storage2/yhaoyuan/transformer_I2S/saved_model/I2U/trimmed_mapping_SpeakerALL_Color/Trial_1/bleu-4_BEST_checkpoint_coco_2_cap_per_img_1_min_word_freq_gpu.pth.tar
# checkpoint_path = "../../saved_model/I2U/trimmed_mapping_SpeakerALL/Trial_3/"
# checkpoint_path = "../../saved_model/I2U/trimmed_mapping_SpeakerALL_Color/Trial_1/"
checkpoint_path = "../../saved_model/I2U/VC_5_captions/Trial_1/"

with open(checkpoint_path + 'config.yml', 'r') as yml:
    config = yaml.safe_load(yml)

checkpoint = checkpoint_path + f'bleu-4_BEST_checkpoint_coco_{str(config["i2u"]["captions_per_image"])}_cap_per_img_{str(config["i2u"]["min_word_freq"])}_min_word_freq_gpu.pth.tar'
dir_name = config["i2u"]["dir_name"]
model_params = config["i2u"]["model_params"]
train_params = config["i2u"]["train_params"]
data_folder = f'../../data/processed/{dir_name}/'  # folder with data files saved by create_input_files.py
# data_name = 'coco_4_cap_per_img_5_min_word_freq'  # base name shared by data files
#data_name = f'coco_{str(config["i2u"]["captions_per_image"])}_cap_per_img_{str(config["i2u"]["min_word_freq"])}_min_word_freq'  # base name shared by data files
data_name = f'coco_{str(config["i2u"]["captions_per_image"])}_cap_per_img_{str(config["i2u"]["min_word_freq"])}_min_word_freq'  # base name shared by data files


In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
# device = torch.device("cpu")

In [29]:
checkpoint = torch.load(checkpoint, map_location=device)

In [30]:
#checkpoint["model_state_dict"]["LM_decoder.layers.0.self_attn.in_proj_weight"].shape

torch.Size([3072, 1024])

In [32]:
for k,v in checkpoint["model_state_dict"].items():
    print(k)

embed.weight
pos_encoder.pe
LM_decoder.layers.0.self_attn.in_proj_weight
LM_decoder.layers.0.self_attn.in_proj_bias
LM_decoder.layers.0.self_attn.out_proj.weight
LM_decoder.layers.0.self_attn.out_proj.bias
LM_decoder.layers.0.linear1.weight
LM_decoder.layers.0.linear1.bias
LM_decoder.layers.0.linear2.weight
LM_decoder.layers.0.linear2.bias
LM_decoder.layers.0.norm1.weight
LM_decoder.layers.0.norm1.bias
LM_decoder.layers.0.norm2.weight
LM_decoder.layers.0.norm2.bias
LM_decoder.layers.1.self_attn.in_proj_weight
LM_decoder.layers.1.self_attn.in_proj_bias
LM_decoder.layers.1.self_attn.out_proj.weight
LM_decoder.layers.1.self_attn.out_proj.bias
LM_decoder.layers.1.linear1.weight
LM_decoder.layers.1.linear1.bias
LM_decoder.layers.1.linear2.weight
LM_decoder.layers.1.linear2.bias
LM_decoder.layers.1.norm1.weight
LM_decoder.layers.1.norm1.bias
LM_decoder.layers.1.norm2.weight
LM_decoder.layers.1.norm2.bias
LM_decoder.layers.2.self_attn.in_proj_weight
LM_decoder.layers.2.self_attn.in_proj_bias


In [5]:
for key, value in model.LM_decoder.state_dict().items():
    #for p in layer.modules():
    print(key, value.shape)
    #break

layers.0.self_attn.in_proj_weight torch.Size([3072, 1024])
layers.0.self_attn.in_proj_bias torch.Size([3072])
layers.0.self_attn.out_proj.weight torch.Size([1024, 1024])
layers.0.self_attn.out_proj.bias torch.Size([1024])
layers.0.linear1.weight torch.Size([4096, 1024])
layers.0.linear1.bias torch.Size([4096])
layers.0.linear2.weight torch.Size([1024, 4096])
layers.0.linear2.bias torch.Size([1024])
layers.0.norm1.weight torch.Size([1024])
layers.0.norm1.bias torch.Size([1024])
layers.0.norm2.weight torch.Size([1024])
layers.0.norm2.bias torch.Size([1024])
layers.1.self_attn.in_proj_weight torch.Size([3072, 1024])
layers.1.self_attn.in_proj_bias torch.Size([3072])
layers.1.self_attn.out_proj.weight torch.Size([1024, 1024])
layers.1.self_attn.out_proj.bias torch.Size([1024])
layers.1.linear1.weight torch.Size([4096, 1024])
layers.1.linear1.bias torch.Size([4096])
layers.1.linear2.weight torch.Size([1024, 4096])
layers.1.linear2.bias torch.Size([1024])
layers.1.norm1.weight torch.Size([10

In [6]:
for key, value in model.decoder.state_dict().items():
    #for p in layer.modules():
    print(key, value.shape)
    #break

layers.0.self_attn.in_proj_weight torch.Size([3072, 1024])
layers.0.self_attn.in_proj_bias torch.Size([3072])
layers.0.self_attn.out_proj.weight torch.Size([1024, 1024])
layers.0.self_attn.out_proj.bias torch.Size([1024])
layers.0.multihead_attn.in_proj_weight torch.Size([3072, 1024])
layers.0.multihead_attn.in_proj_bias torch.Size([3072])
layers.0.multihead_attn.out_proj.weight torch.Size([1024, 1024])
layers.0.multihead_attn.out_proj.bias torch.Size([1024])
layers.0.linear1.weight torch.Size([4096, 1024])
layers.0.linear1.bias torch.Size([4096])
layers.0.linear2.weight torch.Size([1024, 4096])
layers.0.linear2.bias torch.Size([1024])
layers.0.norm1.weight torch.Size([1024])
layers.0.norm1.bias torch.Size([1024])
layers.0.norm2.weight torch.Size([1024])
layers.0.norm2.bias torch.Size([1024])
layers.0.norm3.weight torch.Size([1024])
layers.0.norm3.bias torch.Size([1024])
layers.1.self_attn.in_proj_weight torch.Size([3072, 1024])
layers.1.self_attn.in_proj_bias torch.Size([3072])
layers

### 注意：
encoder里的`norm2` 实际上是decoder里的`norm3`

encoder里的`dropout2` 实际上是decoder里的`dropout3`

所以直接 用 `load_state_dict(dict, strict = False)` 是不行的


In [25]:
decoder_state_dict = model.decoder.state_dict()
model.LM_decoder.load_state_dict(decoder_state_dict, strict=False)

AttributeError: 'NoneType' object has no attribute 'load_state_dict'

In [8]:
LM_decoder_state_dict = model.LM_decoder.state_dict()
model.decoder.load_state_dict(LM_decoder_state_dict, strict=False)

_IncompatibleKeys(missing_keys=['layers.0.multihead_attn.in_proj_weight', 'layers.0.multihead_attn.in_proj_bias', 'layers.0.multihead_attn.out_proj.weight', 'layers.0.multihead_attn.out_proj.bias', 'layers.0.norm3.weight', 'layers.0.norm3.bias', 'layers.1.multihead_attn.in_proj_weight', 'layers.1.multihead_attn.in_proj_bias', 'layers.1.multihead_attn.out_proj.weight', 'layers.1.multihead_attn.out_proj.bias', 'layers.1.norm3.weight', 'layers.1.norm3.bias', 'layers.2.multihead_attn.in_proj_weight', 'layers.2.multihead_attn.in_proj_bias', 'layers.2.multihead_attn.out_proj.weight', 'layers.2.multihead_attn.out_proj.bias', 'layers.2.norm3.weight', 'layers.2.norm3.bias', 'layers.3.multihead_attn.in_proj_weight', 'layers.3.multihead_attn.in_proj_bias', 'layers.3.multihead_attn.out_proj.weight', 'layers.3.multihead_attn.out_proj.bias', 'layers.3.norm3.weight', 'layers.3.norm3.bias', 'layers.4.multihead_attn.in_proj_weight', 'layers.4.multihead_attn.in_proj_bias', 'layers.4.multihead_attn.out_p

In [14]:
from fairseq import options

In [16]:
parser = options.get_interactive_generation_parser()

In [2]:
import time
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import TransformerLM, TransformerConditionedLM, TransformerSentenceLM
from datasets import *
from utils_LM import *       #changed
from nltk.translate.bleu_score import corpus_bleu
# from torcheval.metrics.text import Perplexity
# from torch.optim.lr_scheduler import LambdaLR
import shutil

from torch.utils.tensorboard import SummaryWriter

import yaml
config_path = "../../config_LM.yml"
with open(config_path, 'r') as yml:
    config = yaml.safe_load(yml)

dir_name = config["i2u"]["dir_name"]
model_params = config["i2u"]["model_params"]
train_params = config["i2u"]["train_params"]

# Data parameters
# data_folder = '/media/ssd/caption data'  # folder with data files saved by create_input_files.py
data_folder = f'../../data/processed/{dir_name}/'  # folder with data files saved by create_input_files.py
# data_name = 'coco_4_cap_per_img_5_min_word_freq'  # base name shared by data files
#data_name = f'coco_{str(config["i2u"]["captions_per_image"])}_cap_per_img_{str(config["i2u"]["min_word_freq"])}_min_word_freq'  # base name shared by data files
data_name = f'coco_{str(config["i2u"]["captions_per_image"])}_cap_per_img_{str(config["i2u"]["min_word_freq"])}_min_word_freq'  # base name shared by data files

# Model parameters
# emb_dim = 512  # dimension of word embeddings
# attention_dim = 512  # dimension of attention linear layers
# decoder_dim = 512  # dimension of decoder RNN
# dropout = 0.5
# import torch
# print(torch.__version__)
# print(torch.version.cuda)
# print(torch.cuda.is_available()) #查看cuda是否可用)
# print(torch.backends.cudnn.version())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
# device = torch.device("cpu")
print("Training on device {}".format(device.type))
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# Training parameters
start_epoch = 0
epochs = train_params["epoch"]  # number of epochs to train for (if early stopping is not triggered)
batch_size = train_params["batch_size"]
workers = train_params["num_workers"]  # for data-loading; right now, only 1 works with h5py
# encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
# decoder_lr = 4e-4  # learning rate for decoder
grad_clip = train_params["grad_clip"]  # clip gradients at an absolute value of
best_bleu4 = 0.  # BLEU-4 score right now
best_accuracy = 0.
best_perplexity = 100000
print_freq = train_params["print_freq"]  # print training/validation stats every __ batches
# fine_tune_encoder = model_params["fine_tune_image_encoder"]  # fine-tune encoder?
checkpoint = train_params["checkpoint_path"]  # path to checkpoint, None if none
#checkpoint = "/net/papilio/storage2/yhaoyuan/LAbyLM/model/I2U/gtts_3_captions/BEST_checkpoint_coco_3_cap_per_img_1_min_word_freq_gpu.pth.tar"
use_scheduler = train_params["use_scheduler"]

kl_weight = train_params["kl_weight"]

import sys
is_debug = True if sys.gettrace() else False
if is_debug:
    print("Debugging Mode")
    train_ID = "debugging_sentence"
else:
    print("Training mode")
    train_ID = str(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) )[2:].replace(" ", "_") + "_sentence"
# train_ID = "debugging"

# def get_lr_schedule(optimizer, num_warmup_epochs: int = 10, d_model: int = 2048):
#     def lr_lambda(current_epoch: int):
#         """
#         Eq. (3) in [Transformer paper](https://arxiv.org/abs/1706.03762)
#         """
#         return d_model**(-0.5) * min((current_epoch+1)**(-0.5), (current_epoch+1)*num_warmup_epochs**(-1.5))

#     return LambdaLR(optimizer, lr_lambda, verbose=True)


"""
Training and validation.
"""

global best_bleu4, best_accuracy, best_perplexity, checkpoint, start_epoch, data_name, word_map, device

# Read word map
word_map_file = os.path.join(data_folder, 'WORDMAP_' + data_name + '.json')
with open(word_map_file, 'r') as j:
    word_map = json.load(j)

# Initialize / load checkpoint
model_params['vocab_size'] = len(word_map)
model = TransformerLM(**model_params)

# optimizer = torch.optim.Adam(model.parameters(), train_params["lr"])
optimizer = getattr(torch.optim, train_params["optimizer"])(model.parameters(), lr=train_params["lr"])

# scheduel LR
# if use_scheduler:
#     scheduler = get_lr_schedule(optimizer, train_params["warmup_epoch"], model_params["d_model"])
if checkpoint is not None:
    model, optimizer, start_epoch, best_bleu4, best_accuracy, best_perplexity = load_checkpoint(checkpoint, model, optimizer, device)

if use_scheduler:
    scheduler = get_lr_schedule(optimizer, train_params["warmup_epoch"], model_params["d_model"])


Training on device cuda
Training mode
Loading checkpoint from /net/papilio/storage2/yhaoyuan/transformer_I2S/saved_model/LM/SpokenCOCO_LibriSpeech/23-02-15_13:53:27_sentence/checkpoint_coco_1_cap_per_img_1_min_word_freq.pth.tar
Adjusting learning rate of group 0 to 4.9411e-07.


In [6]:
scheduler.last_epoch = start_epoch

In [8]:
scheduler.get_lr()

/net/papilio/storage2/yhaoyuan/anaconda3/envs/encodec/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


[3.4938562148434215e-06]